### This is first load on dataframe "jpx-tokyo-stock-exchange-prediction/stock_list.csv'"
##### Overall dataset of unique stocks in the data

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('../../jpx-tokyo-stock-exchange-prediction/stock_list.csv')
df.rename(columns={'Section/Products': 'Section', '33SectorName':'Sector_one', '17SectorName':'Sector_two'}, inplace=True)
df['TradeDate'] = pd.to_datetime(df['TradeDate'], unit='s')

In [ ]:
print(f'We have {len(df.SecuritiesCode.unique())} unique stocks in our dataset all with metadata in 15 additional columns : {df.shape[0]} x {df.shape[1]}.')
print(f'We have {len(df.Section.unique())} unique sections in our dataset.')
print(f'We have {len(df.NewMarketSegment.unique())} unique market segments in our dataset.')
print(f'We have {len(df.Sector_one.unique())} unique 33 Sector Names in our dataset.Some are None values, for example : {list(df.Sector_one.unique())[0:5]}')
print(f'We have {len(df.Sector_two.unique())} unique 17 Sector Names in our dataset.Some are None values, for example : {list(df.Sector_two.unique())[0:5]}')

### Loading train_files/stock_prices.csv that is the main stock trading data

In [ ]:
df_sp = pd.read_csv('../../jpx-tokyo-stock-exchange-prediction/train_files/stock_prices.csv')

In [ ]:
df_sp.head(5)

In [ ]:
print(f'We have {len(df_sp.RowId.unique())} unique stocks in our dataset that is indexed by date and security code')
